In [53]:
import pandas as pd

In [54]:
#读取并查看数据格式
train = pd.read_csv("/kaggle/input/labeledtraindata/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(train.shape)
print(train.columns)

(25000, 3)
Index(['id', 'sentiment', 'review'], dtype='object')


In [55]:
#查看一条数据
print(train["id"][0])
print(train["sentiment"][0])
#print(train["review"][0])

"5814_8"
1


**Data clean and Text preprocessing**

In [56]:
#去除.html后缀
#from bs4 import BeautifulSoup
#example1 = BeautifulSoup(train['review'][0])
#print(example1.get_text())

In [57]:
#去除标点符号和非法字符:NLTK 正则表达式
#import re
#letters_only = re.sub("[^a-zA-Z]"," ",example1.get_text()) #非a-zA-Z字符用空格替换
#print(letters_only)
#lower_case = letters_only.lower() #转小写
#print(lower_case)
#words = lower_case.split() #拆分
#print(words)

In [58]:
#去停用词
#import nltk
#from nltk.corpus import stopwords #导入停用词表
#print(stopwords.words("english"))
# 从"words"中去除停用词
#words = [w for w in words if not w in stopwords.words("english")]
#print(words)

In [59]:
#数据清洗整合为函数
from bs4 import BeautifulSoup #去除html包
import re #去除非法字符包
import nltk #去停用词包
from nltk.corpus import stopwords
def review_to_words(raw_review):
    #去除html后缀
    review_text = BeautifulSoup(raw_review).get_text()
    #去除标点和非法字符
    letters_only = re.sub("[^a-zA-Z]"," ",review_text)
    #转小写并拆分得到词表
    words = letters_only.lower().split()
    #去停用词
    stops = set(stopwords.words("english"))
    meaningful_words=[w for w in words if w not in stops]

    return (" ".join(meaningful_words))

In [60]:
#查看单条数据清洗情况
#clean_review = review_to_words(train["review"][0])
#print(clean_review)

In [61]:
#清洗整个训练集的数据
num_reviews = len(train) #获取长度

clean_train_reviews = []

for i in range(0,num_reviews):
    if((i+1)%1000 == 0):
        print("Review %d of %d\n" %(i+1,num_reviews))  #每1000条数据进行汇报
    clean_train_reviews.append(review_to_words(train["review"][i]))

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [62]:
#print(clean_train_reviews[0])

**Create Features from a Bag of words**

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
#初始化vectorizer
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)
#得到训练集特征矩阵
train_data_features = vectorizer.fit_transform(clean_train_reviews)
#将训练集特征矩阵转化为数组
train_data_features = train_data_features.toarray()

In [69]:
#查看训练数据数组的样子
print(train_data_features.shape)

(25000, 5000)


In [75]:
#查看训练好的词袋模型的词汇
vocab = vectorizer.get_feature_names_out()
print (vocab)

['abandoned' 'abc' 'abilities' ... 'zombie' 'zombies' 'zone']


**Random Forest**

In [81]:
print("training the Random Forest..\n")
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

#让森林拟合训练集 使用the bag of words作为特征和情感标签
forest = forest.fit(train_data_features,train['sentiment'])
print("done!")

training the Random Forest..

done!


**Create a Submission**

In [83]:
#读取测试数据
test = pd.read_csv("/kaggle/input/testdata-tsv/testData.tsv",header=0,delimiter='\t',quoting=3)
print(test.shape)

(25000, 2)


In [86]:
#清洗测试数据
num_reviews = len(test['review'])
clean_test_reviews = []
print("cleaning and parsing the test set movie reviews ..\n")

for i in range(0,num_reviews):
    if((i+1)%1000 == 0):
        print("Review %d of %d \n"%(i+1,num_reviews))
    clean_review = review_to_words(test['review'][i])
    clean_test_reviews.append(clean_review)

cleaning and parsing the test set movie reviews ..

Review 1000 of 25000 

Review 2000 of 25000 

Review 3000 of 25000 

Review 4000 of 25000 

Review 5000 of 25000 

Review 6000 of 25000 

Review 7000 of 25000 

Review 8000 of 25000 

Review 9000 of 25000 

Review 10000 of 25000 

Review 11000 of 25000 

Review 12000 of 25000 

Review 13000 of 25000 

Review 14000 of 25000 

Review 15000 of 25000 

Review 16000 of 25000 

Review 17000 of 25000 

Review 18000 of 25000 

Review 19000 of 25000 

Review 20000 of 25000 

Review 21000 of 25000 

Review 22000 of 25000 

Review 23000 of 25000 

Review 24000 of 25000 

Review 25000 of 25000 



In [ ]:
#print(clean_test_reviews[0])

In [88]:
#得到测试集特征矩阵并转为数组
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [89]:
#预测
result = forest.predict(test_data_features)

In [92]:
#输出并保存
output = pd.DataFrame(data={"id":test["id"],"sentiment":result})
output.to_csv("Bag_of_Words_model.csv",index=False,quoting=3)